<a href="https://colab.research.google.com/github/SHKajal/UNSB-CNN/blob/main/ANNv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_train = "/content/drive/MyDrive/UNSW Datasets/a part of training and testing set/UNSW_NB15_training-set.csv"
path_test = "/content/drive/MyDrive/UNSW Datasets/a part of training and testing set/UNSW_NB15_testing-set.csv"

df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

df = pd.concat([df_train, df_test], axis = 0)

In [ ]:
df.head(5)

In [ ]:
# df.isnull().sum() 

In [ ]:
df[df.label == 0].shape

In [ ]:
df[df.label == 1].shape

In [ ]:
df_0 = df[df.label == 0]
df_1 = df[df.label == 1]

In [ ]:
df_0.reset_index(inplace = True)
df_1.reset_index(inplace = True)

In [ ]:
df_0 = df_0.drop(['index'], axis = 1)

**Not Balacing **

In [ ]:
df_1 = df_1.loc[0 : 93000-1].drop(['index'], axis = 1)

In [ ]:
df_1.shape

In [ ]:
df_0.shape

**Start**

In [ ]:
df = pd.concat([df_0, df_1], axis = 0)

In [ ]:
df = df.drop(['id', 'attack_cat'], axis = 1)

In [ ]:
df

In [ ]:
X = df.drop(['label'], axis = 1)
y = df.label

In [ ]:
X

In [ ]:
y

In [ ]:
X_columns = X.columns
X_num_columns = X.describe().columns
X_cat_columns = [col for col in X_columns if col not in X_num_columns]


print(X_cat_columns)

In [ ]:
# One Hot
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector

# onehot_encoder = OneHotEncoder(drop='first')
# transformer = make_column_transformer((OneHotEncoder(), make_column_selector(dtype_include=object)), remainder = "passthrough")
transformer = make_column_transformer((OneHotEncoder(), X_cat_columns), remainder = "passthrough")
transformed = transformer.fit_transform(X).toarray()

print("DF", X.shape)
print("TF", transformed.shape)
print(X.columns.shape)
print(transformer.get_feature_names_out().shape)

tr_df = pd.DataFrame(transformed, columns = transformer.get_feature_names_out())

print("tr_df shape: ", tr_df.shape)

In [ ]:
X = tr_df

In [ ]:
X.head()

In [ ]:
X = (X - X.min()) / (X.max() - X.min())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle= True)

In [ ]:
# from tensorflow import keras

# model = keras.Sequential([
#     keras.layers.Dense(75, input_shape=(X_train.shape[1],), activation='relu'),
#     keras.layers.Dense(50, activation='relu'),
#     keras.layers.Dense(30, activation='relu'),
#     keras.layers.Dense(10, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid'),
# ])

In [ ]:
# import tensorflow as tf

# model.compile(
#     loss=tf.keras.losses.binary_crossentropy,  
#     optimizer=tf.keras.optimizers.Adam(0.0001),
#     metrics=['accuracy']
# )

In [ ]:
# history = model.fit(X_train,y_train,epochs=20)

In [ ]:
# ev = model.evaluate(X_test, y_test)
# print(ev)

In [ ]:
import math

In [ ]:
from tensorflow import keras

model2 = keras.Sequential([
    keras.layers.Dense(X_train.shape[1]*2, input_shape=(X_train.shape[1],), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*1.5), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*1.5), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*1), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*0.995), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*0.80), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*0.75), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*0.60), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*0.50), activation='relu'),
    keras.layers.Dense(math.floor(X_train.shape[1]*0.40), activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])



import tensorflow as tf

model2.compile(
    loss=tf.keras.losses.binary_crossentropy,  
    optimizer=tf.keras.optimizers.Nadam(0.0001),
    metrics=['binary_accuracy']
)



history = model2.fit(X_train,y_train,epochs=100)

In [ ]:
ev = model2.evaluate(X_test, y_test)
print(ev)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
y_pred = model2.predict(X_test)

print(y_pred)

In [ ]:
model2.predict(X_test).round()

In [ ]:
y_test

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred.round())

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred.round())
print(report)